# Customer Segmentation and CLV Prediction

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import datetime as dt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import joblib
import warnings
import os

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load the dataset
try:
    df = pd.read_excel("Online Retail.xlsx", engine='openpyxl')
    print("Data loaded successfully from Excel file.")
except FileNotFoundError:
    print("Error: 'Online Retail.xlsx' not found.")
    print("Please download the dataset and place it in the same directory as this notebook.")
    df = pd.DataFrame(columns=["InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "UnitPrice", "CustomerID", "Country"])

if not df.empty:
    print(df.head())
    print("\nShape of the data:", df.shape)
    df.info()

## 2. Data Preprocessing

In [ ]:
def preprocess_data(df_raw):
    df = df_raw.copy()
    
    initial_rows = df.shape[0]
    df.dropna(subset=['CustomerID'], inplace=True)
    print(f"Dropped {initial_rows - df.shape[0]} rows with missing CustomerID.")

    df['CustomerID'] = df['CustomerID'].astype(int).astype(str)

    df['IsCancelled'] = df['InvoiceNo'].astype(str).str.startswith('C')
    print(f"Found {df['IsCancelled'].sum()} cancelled order lines.")
    
    df = df[df['Quantity'] > 0]
    print(f"Removed rows with negative or zero quantity.")

    df = df[df['UnitPrice'] > 0]
    print(f"Removed rows with zero or negative unit price.")

    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

    initial_rows = df.shape[0]
    df.drop_duplicates(inplace=True)
    print(f"Dropped {initial_rows - df.shape[0]} duplicate rows.")
    
    print(f"\nCleaned data shape: {df.shape}")
    return df

In [ ]:
if not df.empty:
    df_clean = preprocess_data(df)
    print(df_clean.describe())

## 3. Exploratory Data Analysis (EDA)

In [ ]:
if 'df_clean' in locals() and not df_clean.empty:
    df_time = df_clean.set_index('InvoiceDate').resample('M')['TotalPrice'].sum()
    fig1 = px.line(df_time, x=df_time.index, y='TotalPrice', title='Total Revenue Over Time',
                   labels={'TotalPrice': 'Total Revenue', 'InvoiceDate': 'Month'})
    fig1.show()

    top_countries = df_clean.groupby('Country')['TotalPrice'].sum().nlargest(10).reset_index()
    fig2 = px.bar(top_countries, x='Country', y='TotalPrice', title='Top 10 Countries by Revenue',
                  color='Country')
    fig2.show()
    
    df_clean['Log_UnitPrice'] = np.log1p(df_clean['UnitPrice'])
    df_clean['Log_Quantity'] = np.log1p(df_clean['Quantity'])

    fig3 = make_subplots(rows=1, cols=2, subplot_titles=("Distribution of UnitPrice (Log)", "Distribution of Quantity (Log)"))
    fig3.add_trace(go.Histogram(x=df_clean['Log_UnitPrice'], name='UnitPrice'), row=1, col=1)
    fig3.add_trace(go.Histogram(x=df_clean['Log_Quantity'], name='Quantity'), row=1, col=2)
    fig3.update_layout(title_text="Distributions of Price and Quantity (Log-Transformed)", showlegend=False)
    fig3.show()
    
    top_products = df_clean.groupby('Description')['Quantity'].sum().nlargest(20).reset_index()
    fig4 = px.bar(top_products, x='Quantity', y='Description', title='Top 20 Most Sold Products', orientation='h')
    fig4.update_layout(yaxis={'categoryorder':'total ascending'})
    fig4.show()

### 3.1. Cohort Retention Analysis

In [ ]:
def get_month(x): return dt.datetime(x.year, x.month, 1)

def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    return year * 12 + month

In [ ]:
if 'df_clean' in locals() and not df_clean.empty:
    df_clean['InvoiceMonth'] = df_clean['InvoiceDate'].apply(get_month)
    
    grouping = df_clean.groupby('CustomerID')['InvoiceMonth']
    df_clean['CohortMonth'] = grouping.transform('min')
    
    invoice_month_int = get_date_int(df_clean, 'InvoiceMonth')
    cohort_month_int = get_date_int(df_clean, 'CohortMonth')
    
    df_clean['CohortIndex'] = invoice_month_int - cohort_month_int

    cohort_data = df_clean.groupby(['CohortMonth', 'CohortIndex'])['CustomerID'].apply(pd.Series.nunique).reset_index()
    
    cohort_counts = cohort_data.pivot_table(index='CohortMonth',
                                          columns='CohortIndex',
                                          values='CustomerID')

    cohort_sizes = cohort_counts.iloc[:, 0]
    retention = cohort_counts.divide(cohort_sizes, axis=0)
    retention = retention.round(3) * 100
    
    retention.index = retention.index.strftime('%Y-%m')
    
    plt.figure(figsize=(18, 10))
    sns.heatmap(retention, annot=True, fmt='.1f', cmap='viridis_r', 
                yticklabels=retention.index, 
                cbar_kws={'label': 'Retention Rate (%)'})
    plt.title('Customer Retention Cohort Heatmap', fontsize=20, pad=20)
    plt.xlabel('Months Since First Purchase', fontsize=14)
    plt.ylabel('First Purchase Month (Cohort)', fontsize=14)
    plt.show()

## 4. Feature Engineering (RFM & CLV Features)

In [ ]:
def create_customer_features(df_clean):
    train_df = df_clean[(df_clean['InvoiceDate'] >= '2010-12-01') & (df_clean['InvoiceDate'] < '2011-09-01')]
    
    target_df = df_clean[(df_clean['InvoiceDate'] >= '2011-09-01') & (df_clean['InvoiceDate'] < '2011-12-01')]
    
    print(f"Training data: {train_df.shape[0]} rows")
    print(f"Target data: {target_df.shape[0]} rows")

    snapshot_date = dt.datetime(2011, 9, 1)
    
    customer_features = train_df.groupby('CustomerID').agg(
        Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
        Frequency=('InvoiceNo', 'nunique'),
        Monetary=('TotalPrice', 'sum'),
        FirstPurchaseDate=('InvoiceDate', 'min'),
        NumUniqueProducts=('StockCode', 'nunique')
    ).reset_index()

    customer_features['AOV'] = customer_features['Monetary'] / customer_features['Frequency']
    
    customer_features['Tenure'] = (snapshot_date - customer_features['FirstPurchaseDate']).dt.days
    
    customer_features['PurchaseFrequencyRate'] = customer_features['Frequency'] / (customer_features['Tenure'] + 1)
    
    clv_target = target_df.groupby('CustomerID')['TotalPrice'].sum().reset_index()
    clv_target.rename(columns={'TotalPrice': 'CLV_3M'}, inplace=True)

    clv_df = pd.merge(customer_features, clv_target, on='CustomerID', how='left')
    clv_df['CLV_3M'].fillna(0, inplace=True)
    
    print("\nCustomer-level feature dataframe created:")
    print(clv_df.head())
    return clv_df

In [ ]:
if 'df_clean' in locals() and not df_clean.empty:
    clv_df = create_customer_features(df_clean)
    print(clv_df.describe())

In [ ]:
if 'clv_df' in locals():
    fig = make_subplots(rows=2, cols=2, subplot_titles=("Recency Distribution", 
                                                         "Frequency Distribution", 
                                                         "Monetary Distribution",
                                                         "3M-CLV Target Distribution"))
    fig.add_trace(go.Histogram(x=clv_df['Recency'], name='Recency'), row=1, col=1)
    fig.add_trace(go.Histogram(x=clv_df['Frequency'][clv_df['Frequency'] < 100], name='Frequency (filtered)'), row=1, col=2)
    fig.add_trace(go.Histogram(x=clv_df['Monetary'][clv_df['Monetary'] < 5000], name='Monetary (filtered)'), row=2, col=1)
    fig.add_trace(go.Histogram(x=clv_df['CLV_3M'][clv_df['CLV_3M'] > 0], name='CLV (non-zero)'), row=2, col=2)
    fig.update_layout(title_text="Distributions of Raw RFM & CLV Features (some filtered for visibility)", height=700)
    fig.show()

## 5. RFM Analysis and Segmentation

In [ ]:
def create_rfm_segments(df):
    rfm = df.copy()
    
    rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1]).astype(int)
    
    rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]).astype(int)
    rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5]).astype(int)
    
    rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
    
    segment_map = {
        r'[4-5][4-5][4-5]': 'Champions',
        r'[3-5][3-5][1-3]': 'Loyal Customers',
        r'[4-5][1-3][4-5]': 'Potential Loyalists (High Spend, Low Freq)',
        r'[4-5][1-3][1-3]': 'New Customers',
        r'[3-5][1-3][1-3]': 'Promising (Recent, Low Freq/Spend)',
        r'[3-4][4-5][1-5]': 'Loyal Customers (Needs Attention)',
        r'[2-3][2-3][2-3]': 'Needs Attention',
        r'[2-3][1-5][4-5]': 'At Risk (High Spend, Slipping)',
        r'[1-2][3-5][3-5]': 'At Risk (High Value, Not Recent)',
        r'[1-2][1-2][3-5]': 'Hibernating (High Spend, Lost)',
        r'[1-2][1-5][1-2]': 'Lost'
    }
    
    rfm['Segment'] = rfm['RFM_Score'].replace(segment_map, regex=True)
    
    rfm['Segment'] = rfm['Segment'].apply(lambda x: 'Lost' if x.isdigit() else x)

    return rfm

In [ ]:
if 'clv_df' in locals():
    rfm_df = create_rfm_segments(clv_df)
    print(rfm_df[['CustomerID', 'RFM_Score', 'Segment']].head())

### 5.1. Visualizing RFM Segments

In [ ]:
if 'rfm_df' in locals():
    segment_counts = rfm_df['Segment'].value_counts().reset_index()
    segment_counts.columns = ['Segment', 'Count']
    
    fig1 = px.bar(segment_counts, x='Segment', y='Count', color='Segment', 
                  title='Customer Segment Distribution')
    fig1.update_layout(xaxis={'categoryorder':'total descending'})
    fig1.show()

    fig2 = px.scatter(rfm_df, x='Recency', y='Frequency', color='Segment', 
                      title='Recency vs. Frequency by Segment', 
                      hover_data=['CustomerID', 'Monetary', 'Segment'],
                      log_y=True, log_x=True, 
                      color_discrete_sequence=px.colors.qualitative.Plotly)
    fig2.show()

In [ ]:
if 'rfm_df' in locals():
    segment_summary = rfm_df.groupby('Segment').agg(
        Recency=('Recency', 'mean'),
        Frequency=('Frequency', 'mean'),
        Monetary=('Monetary', 'mean'),
        CLV_3M=('CLV_3M', 'mean'),
        Count=('CustomerID', 'count')
    ).reset_index()

    scaler = MinMaxScaler()
    metrics_to_plot = ['Recency', 'Frequency', 'Monetary', 'CLV_3M']
    summary_scaled = segment_summary.copy()
    summary_scaled[metrics_to_plot] = scaler.fit_transform(summary_scaled[metrics_to_plot])
    summary_scaled['Recency'] = 1 - summary_scaled['Recency']
    
    fig3 = go.Figure()
    
    for i, row in summary_scaled.iterrows():
        fig3.add_trace(go.Scatterpolar(
            r=row[metrics_to_plot].values,
            theta=metrics_to_plot,
            fill='toself',
            name=row['Segment']
        ))

    fig3.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        ),
        showlegend=True,
        title="Normalized Segment Characteristics (Recency is inverted)"
    )
    fig3.show()

## 6. Customer Lifetime Value (CLV) Prediction

In [ ]:
if 'rfm_df' in locals():
    features = ['Recency', 'Frequency', 'Monetary', 'AOV', 'Tenure', 'NumUniqueProducts', 'PurchaseFrequencyRate']
    target = 'CLV_3M'

    X = rfm_df[features]
    y = rfm_df[target]

    y_log = np.log1p(y)

    X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print(f"Training features shape: {X_train_scaled.shape}")
    print(f"Test features shape: {X_test_scaled.shape}")

In [ ]:
def evaluate_model(y_true_log, y_pred_log):
    y_true = np.expm1(y_true_log)
    y_pred = np.expm1(y_pred_log)
    
    y_pred[y_pred < 0] = 0
    
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    return r2, mae, rmse

### 6.1. Model Training

In [ ]:
model_performance = {}

if 'X_train_scaled' in locals():
    print("Training Linear Regression...")
    lr = LinearRegression()
    lr.fit(X_train_scaled, y_train_log)
    y_pred_log_lr = lr.predict(X_test_scaled)
    r2_lr, mae_lr, rmse_lr = evaluate_model(y_test_log, y_pred_log_lr)
    model_performance['Linear Regression'] = {'R2': r2_lr, 'MAE': mae_lr, 'RMSE': rmse_lr}
    print("Linear Regression Done.")

    print("Training Random Forest...")
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=10)
    rf.fit(X_train_scaled, y_train_log)
    y_pred_log_rf = rf.predict(X_test_scaled)
    r2_rf, mae_rf, rmse_rf = evaluate_model(y_test_log, y_pred_log_rf)
    model_performance['Random Forest'] = {'R2': r2_rf, 'MAE': mae_rf, 'RMSE': rmse_rf}
    print("Random Forest Done.")

    print("Training XGBoost...")
    xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', 
                               n_estimators=100, 
                               learning_rate=0.1, 
                               max_depth=5, 
                               random_state=42, 
                               n_jobs=-1)
    xgb_reg.fit(X_train_scaled, y_train_log)
    y_pred_log_xgb = xgb_reg.predict(X_test_scaled)
    r2_xgb, mae_xgb, rmse_xgb = evaluate_model(y_test_log, y_pred_log_xgb)
    model_performance['XGBoost'] = {'R2': r2_xgb, 'MAE': mae_xgb, 'RMSE': rmse_xgb}
    print("XGBoost Done.")

    print("\n--- Model Performance Comparison ---")
    perf_df = pd.DataFrame(model_performance).T
    print(perf_df)

### 6.2. Model Interpretation (Feature Importance)

In [ ]:
if 'xgb_reg' in locals():
    importance = pd.DataFrame({'Feature': features, 'Importance': xgb_reg.feature_importances_})
    importance = importance.sort_values(by='Importance', ascending=False)

    fig = px.bar(importance, x='Importance', y='Feature', orientation='h', 
                 title='Feature Importance for CLV Prediction (XGBoost)')
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    fig.show()

In [ ]:
if 'xgb_reg' in locals():
    y_test_true = np.expm1(y_test_log)
    y_test_pred = np.expm1(y_pred_log_xgb)
    y_test_pred[y_test_pred < 0] = 0

    # FIXED: Use min(1000, len(y_test_true)) to avoid sampling more than available
    sample_size = min(1000, len(y_test_true))
    plot_df = pd.DataFrame({'True CLV': y_test_true, 'Predicted CLV': y_test_pred}).sample(n=sample_size, random_state=42)
    
    fig = px.scatter(plot_df, x='True CLV', y='Predicted CLV', 
                     title=f'True vs. Predicted 3-Month CLV (XGBoost) - Sample of {sample_size}',
                     opacity=0.5,
                     trendline='ols', trendline_color_override='red')
    fig.add_shape(type='line', x0=0, y0=0, x1=plot_df['True CLV'].max(), y1=plot_df['True CLV'].max(), 
                  line=dict(color='blue', dash='dash'), name='Perfect Prediction')
    fig.show()

## 7. Clustering (K-Means)

In [ ]:
if 'rfm_df' in locals():
    cluster_features = ['Recency', 'Frequency', 'Monetary', 'AOV', 'Tenure']
    
    cluster_df = rfm_df[cluster_features].copy()
    cluster_df['Recency'] = np.log1p(cluster_df['Recency'])
    cluster_df['Frequency'] = np.log1p(cluster_df['Frequency'])
    cluster_df['Monetary'] = np.log1p(cluster_df['Monetary'])
    cluster_df['AOV'] = np.log1p(cluster_df['AOV'])
    cluster_df['Tenure'] = np.log1p(cluster_df['Tenure'])
    
    cluster_scaler = StandardScaler()
    cluster_scaled = cluster_scaler.fit_transform(cluster_df)
    
    wcss = []
    k_range = range(2, 11)
    
    print("Running Elbow Method...")
    for k in k_range:
        kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
        kmeans.fit(cluster_scaled)
        wcss.append(kmeans.inertia_)
    print("Elbow Method Done.")

    fig = px.line(x=k_range, y=wcss, title='Elbow Method for Optimal K',
                  labels={'x': 'Number of Clusters (k)', 'y': 'WCSS'})
    fig.add_vline(x=5, line_dash="dash", line_color="red", annotation_text="Optimal k = 5")
    fig.show()

In [ ]:
if 'cluster_scaled' in locals():
    optimal_k = 5
    kmeans = KMeans(n_clusters=optimal_k, init='k-means++', random_state=42, n_init=10)
    rfm_df['KMeans_Cluster'] = kmeans.fit_predict(cluster_scaled)
    
    cluster_centers = kmeans.cluster_centers_
    
    centers_df = pd.DataFrame(cluster_centers, columns=cluster_features)
    centers_df['Cluster'] = centers_df.index
    print("K-Means Cluster Centers (Scaled)")
    print(centers_df)
    
    print("\n--- K-Means vs RFM Segment Crosstab ---")
    crosstab = pd.crosstab(rfm_df['KMeans_Cluster'], rfm_df['Segment'])
    print(crosstab)
    
    rfm_df['KMeans_Cluster_Str'] = 'Cluster ' + rfm_df['KMeans_Cluster'].astype(str)
    cluster_summary = rfm_df.groupby('KMeans_Cluster_Str').agg(
        Recency=('Recency', 'mean'),
        Frequency=('Frequency', 'mean'),
        Monetary=('Monetary', 'mean'),
        CLV_3M=('CLV_3M', 'mean')
    ).reset_index()

## 8. Final Model Saving

In [ ]:
if 'xgb_reg' in locals():
    os.makedirs('outputs/models', exist_ok=True)
    
    model_path = 'outputs/models/clv_model.pkl'
    joblib.dump(xgb_reg, model_path)
    print(f"Model saved to {model_path}")

    scaler_path = 'outputs/models/scaler.pkl'
    joblib.dump(scaler, scaler_path)
    print(f"Scaler saved to {scaler_path}")